In [1]:
"I am blessed" 

["I","am","blessed"] #- Token of words

# [1 0  0 
# 0  1  0
# 0  0  1] * Weight Random matrix 

# Take a list of token of words as a matrics


['I', 'am', 'blessed']

In [2]:
## Word Emdedding using Python 
import numpy as np

# Define the vocabulary
vocab = {"Monika": 0, "likes": 1, "Coffee": 2}


# One-hot encode each word - using numpy -- give index to each words
def one_hot_encoding(word_index, vocab_size):
    one_hot = np.zeros(vocab_size)
    one_hot[word_index] = 1
    return one_hot

# Define the weight matrix (random initialization for simplicity)
weight_matrix = np.random.rand(len(vocab), 3)

# Multiply one-hot encoded words with the weight matrix
word_embeddings = {}
for word, index in vocab.items():
    one_hot_encoded = one_hot_encoding(index, len(vocab))
    embedding = np.dot(one_hot_encoded, weight_matrix)
    word_embeddings[word] = embedding

# Display word embeddings
for word, embedding in word_embeddings.items():
    print(f"{word}: {embedding}")


Monika: [0.5391266  0.28676013 0.68882481]
likes: [0.74886597 0.23254551 0.97288267]
Coffee: [0.05358873 0.17295359 0.08770437]


In [3]:
#Generate positional encodings using sine and cosine functions
import numpy as np

# Position Encoding
def position_encoding(sentence_length, embedding_dim):
    position_encodings = np.zeros((sentence_length, embedding_dim))
    for pos in range(sentence_length):
        for i in range(embedding_dim):
            if i % 2 == 0:
                position_encodings[pos, i] = np.sin(pos / (10000 ** (i / embedding_dim)))  # scale th position encoding 
            else:
                position_encodings[pos, i] = np.cos(pos / (10000 ** ((i - 1) / embedding_dim)))
    return position_encodings

# Assuming we have word embeddings for "Monika", "likes", "Coffee" as follows
word_embeddings = {
    "Monika": np.array([0.1, 0.2, 0.3]),
    "likes": np.array([0.2, 0.3, 0.1]),
    "coffee": np.array([0.3, 0.1, 0.2])
}

# Get the position encodings
sentence_length = 3
embedding_dim = 3
pos_encodings = position_encoding(sentence_length, embedding_dim)

# Add position encodings to word embeddings
for i, word in enumerate(word_embeddings):
    word_embeddings[word] += pos_encodings[i % sentence_length]

# Display word embeddings with position encodings
print("Word embeddings with position encodings:")
for word, embedding in word_embeddings.items():
    print(f"{word}: {embedding}")
# This helps us identify which position in the sentence - we always provide sine functions since it gives the position in the sentence details

Word embeddings with position encodings:
Monika: [0.1 1.2 0.3]
likes: [1.04147098 0.84030231 0.10215443]
coffee: [ 1.20929743 -0.31614684  0.20430886]


In [4]:
# Now we will Implement text generation using models

In [5]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
import torch
from transformers import BertModel, BertTokenizer, BertConfig

# Load pre-trained BERT model and tokenizer
machine_T = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(machine_T )
model = BertModel.from_pretrained(machine_T )

# Input text
input_text = input("enter your text : ")

# Tokenize input text
input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt")

# Get BERT model's output
outputs = model(input_ids)


enter your text :  I love LLMs, they make me happy!


In [7]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3841, -0.0234, -0.2017,  ...,  0.5245, -0.1391,  0.1791],
         [-0.1159, -0.1716,  0.2443,  ...,  0.9176, -0.9874,  0.0983],
         [-0.0056, -0.0829, -0.4190,  ...,  0.9390, -0.7476,  0.7224],
         ...,
         [-0.8440, -0.4264, -0.3613,  ...,  0.8687, -0.2631,  0.2862],
         [-0.7841, -0.0480, -0.4827,  ...,  0.4807, -0.1298,  0.2611],
         [-0.6237, -0.0068,  0.0317,  ...,  0.3305,  0.0342,  0.0811]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.4370, -0.0656,  0.3193, -0.2134, -0.1019,  0.5360,  0.2850,  0.1132,
         -0.3671,  0.3552, -0.0741, -0.2581, -0.2375, -0.0639,  0.1875, -0.3000,
          0.7516,  0.0432,  0.2976, -0.3649, -1.0000, -0.1487, -0.2942, -0.2246,
         -0.3305, -0.0301, -0.2011,  0.1315,  0.2810, -0.2672,  0.2703, -1.0000,
          0.6721,  0.7428,  0.3939, -0.1486,  0.2582,  0.2270,  0.1492, -0.3759,
         -0.0324,  0.0083, -0.26

In [8]:
#Check configurations of model
# Check if the model supports attention weights
config = BertConfig.from_pretrained(machine_T )
if config.output_attentions:
    # Extract hidden states
    hidden_states = outputs.last_hidden_state

    # Self-attention
    self_attention_weights = outputs.attentions

    # Print self-attention weights
    print("Self-attention weights:")
    for layer, attn_weights in enumerate(self_attention_weights):
        print(f"Layer {layer+1}: {attn_weights.mean(dim=1)}")

# Decoding input text
decoded_output = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("Decoded output from positional encoder:", decoded_output)

# Get the model's final output
final_output = outputs[0]

# Print the final output
print("Final output:", final_output)

print("Shape of the final Output",final_output.shape)

Decoded output from positional encoder: I love LLMs, they make me happy!
Final output: tensor([[[-0.3841, -0.0234, -0.2017,  ...,  0.5245, -0.1391,  0.1791],
         [-0.1159, -0.1716,  0.2443,  ...,  0.9176, -0.9874,  0.0983],
         [-0.0056, -0.0829, -0.4190,  ...,  0.9390, -0.7476,  0.7224],
         ...,
         [-0.8440, -0.4264, -0.3613,  ...,  0.8687, -0.2631,  0.2862],
         [-0.7841, -0.0480, -0.4827,  ...,  0.4807, -0.1298,  0.2611],
         [-0.6237, -0.0068,  0.0317,  ...,  0.3305,  0.0342,  0.0811]]],
       grad_fn=<NativeLayerNormBackward0>)
Shape of the final Output torch.Size([1, 13, 768])


## Example Language Translation

In [9]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
import torch
from transformers import MarianMTModel, MarianTokenizer

# Load pre-trained MarianMT model and tokenizer for English to Hindi translation
model_name = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(model_name)
hindi_tran = MarianMTModel.from_pretrained(model_name)

# Input text
input_text = input("Enter text for transalation = ")
print("Input=",input_text)

# Tokenize input text
input_ids = tokenizer(input_text, return_tensors="pt")

# Perform translation
translated_output = hindi_tran.generate(**input_ids)

# Decode the translated output
translated_text = tokenizer.decode(translated_output[0], skip_special_tokens=True)

# Print the translated output
print("Translated Output (Hindi):")
print(translated_text)

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

Enter text for transalation =  I am fine and happy


Input= I am fine and happy
Translated Output (Hindi):
मैं ठीक हूँ और खुश हूँ


In [16]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
from transformers import MarianMTModel, MarianTokenizer

# Load the pre-trained translation model and tokenizer for English to German
model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Input text in English
input_text = input("Enter the Text: ")

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Perform translation
outputs = model.generate(**inputs)

# Decode the translated text
decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the decoded text
print("Decoded text in German:", decoded_text)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

Enter the Text:  I am Sajag


Decoded text in German: Ich bin Sajag.


## Hindi translation

In [17]:
from transformers import MarianMTModel, MarianTokenizer
import torch

# Define the model and tokenizer for English to Hindi
model_name_en_to_hi = 'Helsinki-NLP/opus-mt-en-hi'
model_en_to_hi = MarianMTModel.from_pretrained(model_name_en_to_hi)
tokenizer_en_to_hi = MarianTokenizer.from_pretrained(model_name_en_to_hi)

def translate_en_to_hi(text):
    inputs = tokenizer_en_to_hi(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        translated_tokens = model_en_to_hi.generate(**inputs)
    return tokenizer_en_to_hi.batch_decode(translated_tokens, skip_special_tokens=True)[0]

english_text = "Hello, how are you?"
hindi_text = translate_en_to_hi(english_text)
print(f'Hindi Translation: {hindi_text}')



Hindi Translation: हैलो, तुम कैसे हो?


In [19]:
pip install wordcloud --trusted-host pypi.org --trusted-host files.pythonhosted.org pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


## Sentiment Analysis

-  We are using distilbert-base-uncased-finetuned-sst-2-english model, which is fine-tuned for sentiment analysis on the Stanford Sentiment Treebank (SST-2) dataset.

In [26]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.eval()

# Define labels
labels = ['Negative', 'Positive']

# Function to perform text classification
def classify_text(text):
    # Tokenize input text
    # Padding - You apply uniform length for tokens so that you get good results. Similar to images where you make padding to reduce loss
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Classify text using BERT model
    with torch.no_grad():
        outputs = model(**inputs)
    # Get predicted probabilities
    probabilities = torch.softmax(outputs.logits, dim=1).squeeze().numpy()
    # Get predicted label index
    predicted_label_index = np.argmax(probabilities)
    # Get predicted label
    predicted_label = labels[predicted_label_index]
    return predicted_label, probabilities

# User input for text
user_text = input("Enter the text you want to classify: ")

# Perform text classification
predicted_label, probabilities = classify_text(user_text)

# Print results
print("Predicted label:", predicted_label)
print("Probabilities:", probabilities)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter the text you want to classify:  I am happy and sad as well. I don't know what to be positive or negative


Predicted label: Positive
Probabilities: [0.326862 0.673138]
